# DataSet #2: Mobilitat entre Municipis

Autor: Marc Bosch Manzano<br>
Data: Octubre 2025<br>
Iteració: 2a<br>
Projecte: MobiCat

## Estructuració de les Dades

In [15]:
import importlib
import sys
import os
sys.path.append('../mobicat_python_utils')

import utils
importlib.reload(utils)
utils.install_missing_packages(["pandas"])

import pandas as pd 
import numpy as np
import IPython.display 
IPython.display.clear_output()

## Agrupaments de dades

- Dades de cronologia: day, day_of_week, month
- Dades d'origen dels visiants: origen
- Dades de municipis: municipio_origen, municipio_origen_name, municipio_destino, municipio_destino_name

### Agrupaments per cronologia

In [2]:
all_datasets_directory = r"F:\mobile_challenge_telefonica\output_limite_legal"
all_datasets_directory_filtered = r"F:\mobile_challenge_telefonica\municipios_filtered"
test_data_directory = r".\test-data"

file_names = [
    "movilidad_municipios_date_sum.csv",
    "movilidad_municipios_year_month_sum.csv",
    "movilidad_municipios_year_week_average.csv",
    "movilidad_municipios_day_of_week_average.csv",
    "movilidad_municipios_month_average.csv",
    "movilidad_municipios_day_average.csv",
    "movilidad_municipios_date_visitor.csv",
    "datasets_names.csv"
]

file_paths = [os.path.join(test_data_directory, f) for f in file_names]

In [3]:
datasets_names = utils.get_datasets_names(all_datasets_directory)
datasets_names_df = utils.get_datasets_names_df(all_datasets_directory)
datasets_names_df.groupby(["type"])["size_in_bytes"].sum().reset_index()

,type,size_in_bytes
0,barrios,2063316513
1,mun_barrios,9125482648
2,municipios,5094513501


In [4]:
try:
    datasets_names_df = pd.read_csv(r".\test-data\datasets_names.csv")
except:
    datasets_names_df = utils.get_datasets_names(all_datasets_directory)
    datasets_names_df.to_csv(r".\test-data\datasets_names.csv", index=False)
datasets_names_municipios_df = datasets_names_df[datasets_names_df["type"] == "municipios"].drop(columns="type").reset_index()

In [5]:
filtered_files_directory = r"F:\mobile_challenge_telefonica\municipios_filtered"
paths = []
for file in os.listdir(filtered_files_directory):
    paths.append({"path": os.path.join(filtered_files_directory, file)})
paths_df = pd.DataFrame(paths)
paths_df.head(5)
# utils.full_datasets_filter_zeros(datasets_names_municipios_df)

,path
0,F:\mobile_challenge_telefonica\municipios_filt...
1,F:\mobile_challenge_telefonica\municipios_filt...
2,F:\mobile_challenge_telefonica\municipios_filt...
3,F:\mobile_challenge_telefonica\municipios_filt...
4,F:\mobile_challenge_telefonica\municipios_filt...


### Agrupaments per data, municipi origen, municipi destí dirigit

In [6]:
temp_file_path = r"F:\mobile_challenge_telefonica\municipios_processed\day_municipio_origen_destino.csv"
try:
    day_municipio_origen_destino_df = pd.read_csv(temp_file_path)
except:
    day_municipio_origen_destino_df = utils.full_datasets_groupby(paths_df, by=["day", "municipio_origen", "municipio_destino"])
    day_municipio_origen_destino_df.to_csv(temp_file_path, index=False)
day_municipio_origen_destino_df

,day,municipio_origen,municipio_destino,viajes
0,2023-01-01,8001,8015,52
1,2023-01-01,8001,8019,920
2,2023-01-01,8001,8025,95
3,2023-01-01,8001,8044,28
4,2023-01-01,8001,8053,41
...,...,...,...,...
11108048,2025-08-31,8905,8263,50
11108049,2025-08-31,8905,8270,16
11108050,2025-08-31,8905,8295,85
11108051,2025-08-31,8905,8301,16


In [7]:
day_municipio_origen_destino_df["day"] = pd.to_datetime(day_municipio_origen_destino_df["day"])
day_extended_municipio_origen_destino_df = day_municipio_origen_destino_df.copy()
day_extended_municipio_origen_destino_df["month"] = day_extended_municipio_origen_destino_df["day"].dt.month
day_extended_municipio_origen_destino_df["year"] = day_extended_municipio_origen_destino_df["day"].dt.year
day_extended_municipio_origen_destino_df["day"] = day_extended_municipio_origen_destino_df["day"].dt.day
day_extended_municipio_origen_destino_df = day_extended_municipio_origen_destino_df[
    ["year", "month", "day", "municipio_origen", "municipio_destino", "viajes"]
]
day_extended_municipio_origen_destino_df

,year,month,day,municipio_origen,municipio_destino,viajes
0,2023,1,1,8001,8015,52
1,2023,1,1,8001,8019,920
2,2023,1,1,8001,8025,95
3,2023,1,1,8001,8044,28
4,2023,1,1,8001,8053,41
...,...,...,...,...,...,...
11108048,2025,8,31,8905,8263,50
11108049,2025,8,31,8905,8270,16
11108050,2025,8,31,8905,8295,85
11108051,2025,8,31,8905,8301,16


In [12]:
temp_file_path = r".\test-data\month_municipio_origen_destino.csv"
try:
    month_municipio_origen_destino_df = pd.read_csv(temp_file_path)
except:
    month_municipio_origen_destino_df = day_extended_municipio_origen_destino_df\
        .groupby(["year", "month", "municipio_origen", "municipio_destino"])["viajes"].sum().reset_index()
    month_municipio_origen_destino_df.to_csv(temp_file_path, index=False)
month_municipio_origen_destino_df

,year,month,municipio_origen,municipio_destino,viajes
0,2023,1,8001,8005,34
1,2023,1,8001,8008,17
2,2023,1,8001,8015,3238
3,2023,1,8001,8019,53208
4,2023,1,8001,8020,15
...,...,...,...,...,...
770059,2025,8,8905,8300,16
770060,2025,8,8905,8301,538
770061,2025,8,8905,8305,162
770062,2025,8,8905,8307,17


In [22]:
temp_file_path = r".\test-data\year_municipio_origen_destino.csv"
try:
    year_municipio_origen_destino_df = pd.read_csv(temp_file_path)
except:
    year_municipio_origen_destino_df = month_municipio_origen_destino_df\
        .groupby(["year", "municipio_origen", "municipio_destino"])["viajes"].sum().reset_index()
    year_municipio_origen_destino_df.to_csv(temp_file_path, index=False)
year_municipio_origen_destino_df

,year,municipio_origen,municipio_destino,viajes
0,2023,8001,8003,57
1,2023,8001,8005,144
2,2023,8001,8006,34
3,2023,8001,8008,17
4,2023,8001,8009,18
...,...,...,...,...
121984,2025,8905,8301,4719
121985,2025,8905,8304,36
121986,2025,8905,8305,2200
121987,2025,8905,8307,681


In [14]:
temp_file_path = r".\test-data\municipio_origen_destino.csv"
try:
    municipio_origen_destino_df = pd.read_csv(temp_file_path)
except:
    municipio_origen_destino_df = year_municipio_origen_destino_df\
        .groupby(["municipio_origen", "municipio_destino"])["viajes"].sum().reset_index()
    municipio_origen_destino_df.to_csv(temp_file_path, index=False)
municipio_origen_destino_df

,municipio_origen,municipio_destino,viajes
0,8001,8002,45
1,8001,8003,231
2,8001,8005,361
3,8001,8006,254
4,8001,8007,31
...,...,...,...
51243,8905,8304,36
51244,8905,8305,9101
51245,8905,8307,1798
51246,8905,8902,26


### Agrupaments per data, municipi origen, municipi destí no dirigit

In [19]:
temp_file_path = r".\test-data\day_municipio_origen_destino_undirected.csv"
try:
    day_extended_municipio_origen_destino_undirected_df = pd.read_csv(temp_file_path)
except:
    df = day_extended_municipio_origen_destino_df.copy()

    municipios = np.sort(df[['municipio_origen', 'municipio_destino']], axis=1)
    df['municipio_1'] = municipios[:, 0]
    df['municipio_2'] = municipios[:, 1]

    day_extended_municipio_origen_destino_undirected_df = df[
        ['year', 'month', 'day', 'municipio_1', 'municipio_2', 'viajes']
    ].copy()
    day_extended_municipio_origen_destino_undirected_df = day_extended_municipio_origen_destino_undirected_df\
        .groupby(['year', 'month', 'day', 'municipio_1', 'municipio_2'])["viajes"].sum().reset_index()
    day_extended_municipio_origen_destino_undirected_df

    day_extended_municipio_origen_destino_undirected_df.to_csv(temp_file_path, index=False)
day_extended_municipio_origen_destino_undirected_df

,year,month,day,municipio_1,municipio_2,viajes
0,2023,1,1,8001,8015,136
1,2023,1,1,8001,8019,1696
2,2023,1,1,8001,8025,195
3,2023,1,1,8001,8044,54
4,2023,1,1,8001,8053,92
...,...,...,...,...,...,...
6338866,2025,8,31,8303,8901,16
6338867,2025,8,31,8304,8305,516
6338868,2025,8,31,8305,8307,4577
6338869,2025,8,31,8305,8904,17


In [21]:
temp_file_path = r".\test-data\month_municipio_origen_destino_undirected.csv"
try:
    month_municipio_origen_destino_undirected_df = pd.read_csv(temp_file_path)
except:
    month_municipio_origen_destino_undirected_df = day_extended_municipio_origen_destino_undirected_df\
        .groupby(["year", "month", "municipio_1", "municipio_2"])["viajes"].sum().reset_index()
    month_municipio_origen_destino_undirected_df.to_csv(temp_file_path, index=False)
month_municipio_origen_destino_undirected_df

,year,month,municipio_1,municipio_2,viajes
0,2023,1,8001,8003,17
1,2023,1,8001,8005,156
2,2023,1,8001,8006,19
3,2023,1,8001,8008,17
4,2023,1,8001,8015,6950
...,...,...,...,...,...
438738,2025,8,8307,8902,19
438739,2025,8,8307,8904,228
438740,2025,8,8307,8905,34
438741,2025,8,8902,8904,24


In [23]:
temp_file_path = r".\test-data\year_municipio_origen_destino_undirected.csv"
try:
    year_municipio_origen_destino_undirected_df = pd.read_csv(temp_file_path)
except:
    year_municipio_origen_destino_undirected_df = month_municipio_origen_destino_undirected_df\
        .groupby(["year", "municipio_1", "municipio_2"])["viajes"].sum().reset_index()
    year_municipio_origen_destino_undirected_df.to_csv(temp_file_path, index=False)
year_municipio_origen_destino_undirected_df

,year,municipio_1,municipio_2,viajes
0,2023,8001,8003,107
1,2023,8001,8005,404
2,2023,8001,8006,69
3,2023,8001,8008,151
4,2023,8001,8009,18
...,...,...,...,...
68622,2025,8307,8904,1240
68623,2025,8307,8905,1041
68624,2025,8308,8904,60
68625,2025,8902,8904,254


In [24]:
temp_file_path = r".\test-data\municipio_origen_destino_undirected.csv"
try:
    municipio_origen_destino_undirected_df = pd.read_csv(temp_file_path)
except:
    municipio_origen_destino_undirected_df = year_municipio_origen_destino_undirected_df\
        .groupby(["municipio_1", "municipio_2"])["viajes"].sum().reset_index()
    municipio_origen_destino_undirected_df.to_csv(temp_file_path, index=False)
municipio_origen_destino_undirected_df

,municipio_1,municipio_2,viajes
0,8001,8002,62
1,8001,8003,420
2,8001,8005,837
3,8001,8006,524
4,8001,8007,47
...,...,...,...
28370,8307,8905,3157
28371,8308,8904,60
28372,8902,8904,1635
28373,8902,8905,26


### Agrupament per data i origen, data i destí, i data i origen-destí combinat

In [ ]:
day_municipio_origen_df = day_extended_municipio_origen_destino_df\
    .groupby(["year", "month", "day", "municipio_origen"])["viajes"].sum().reset_index()

day_municipio_destino_df = day_extended_municipio_origen_destino_df\
    .groupby(["year", "month", "day", "municipio_destino"])["viajes"].sum().reset_index()

day_municipio_df = pd.concat([
    day_municipio_origen_df.rename(columns={"municipio_origen": "municipio"}),
    day_municipio_destino_df.rename(columns={"municipio_destino": "municipio"})
])

day_municipio_df = day_municipio_df\
    .groupby(["year", "month", "day", "municipio"]).sum().reset_index()
day_municipio_df

,year,month,day,municipio,viajes
0,2023,1,1,08001,14042
1,2023,1,1,08002,384
2,2023,1,1,08003,15767
3,2023,1,1,08004,15
4,2023,1,1,08005,7892
...,...,...,...,...,...
288192,2025,8,31,08307,50316
288193,2025,8,31,08901,772
288194,2025,8,31,08902,5421
288195,2025,8,31,08904,5253


In [107]:
month_municipio_origen_df = day_municipio_origen_df\
    .groupby(["year", "month", "municipio_origen"])["viajes"].sum().reset_index()
month_municipio_origen_df

month_municipio_destino_df = day_municipio_destino_df\
    .groupby(["year", "month", "municipio_destino"])["viajes"].sum().reset_index()
month_municipio_destino_df

month_municipio_df = day_municipio_df\
    .groupby(["year", "month", "municipio"])["viajes"].sum().reset_index()
month_municipio_df

,year,month,municipio,viajes
0,2023,1,08001,926039
1,2023,1,08002,31743
2,2023,1,08003,676085
3,2023,1,08004,2064
4,2023,1,08005,339925
...,...,...,...,...
9698,2025,8,08901,20872
9699,2025,8,08902,169394
9700,2025,8,08903,130
9701,2025,8,08904,190521


In [ ]:
year_municipio_origen_df = month_municipio_origen_df\
    .groupby(["year", "municipio_origen"])["viajes"].sum().reset_index()

year_municipio_destino_df = month_municipio_destino_df\
    .groupby(["year", "municipio_destino"])["viajes"].sum().reset_index()

year_municipio_df = month_municipio_df\
    .groupby(["year", "municipio"])["viajes"].sum().reset_index()
year_municipio_df

,year,municipio,viajes
0,2023,08001,11520567
1,2023,08002,656601
2,2023,08003,8269251
3,2023,08004,38385
4,2023,08005,3880677
...,...,...,...
917,2025,08901,84616
918,2025,08902,1794646
919,2025,08903,3919
920,2025,08904,2416622


In [109]:
municipio_origen_df = year_municipio_origen_df\
    .groupby(["municipio_origen"])["viajes"].sum().reset_index()

municipio_destino_df = month_municipio_destino_df\
    .groupby(["municipio_destino"])["viajes"].sum().reset_index()

municipio_df = month_municipio_df\
    .groupby(["municipio"])["viajes"].sum().reset_index()
municipio_df

,municipio,viajes
0,08001,32989069
1,08002,2056068
2,08003,21417707
3,08004,58294
4,08005,10684636
...,...,...
304,08901,209784
305,08902,8601979
306,08903,28178
307,08904,7776563


Tipus de viatger

In [110]:
origen_municipio_origen_destino_df = utils.full_datasets_groupby(paths_df, by=["origen", "municipio_origen", "municipio_destino"])

In [116]:
origen_municipio_origen_df = origen_municipio_origen_destino_df\
    .groupby(["origen", "municipio_origen"])["viajes"].sum().reset_index()

origen_municipio_destino_df = origen_municipio_origen_destino_df\
    .groupby(["origen", "municipio_destino"])["viajes"].sum().reset_index()

origen_municipio_df = pd.concat([
    origen_municipio_origen_df.rename(columns={"municipio_origen": "municipio"}),
    origen_municipio_destino_df.rename(columns={"municipio_destino": "municipio"})
])

origen_municipio_df = origen_municipio_df\
    .groupby(["origen", "municipio"]).sum().reset_index()
origen_municipio_df

,origen,municipio,viajes
0,Internacional,08001,2743410
1,Internacional,08002,243228
2,Internacional,08003,2003046
3,Internacional,08004,146
4,Internacional,08005,279680
...,...,...,...
1197,Residente,08901,13595
1198,Residente,08902,479663
1199,Residente,08903,1017
1200,Residente,08904,108768


In [36]:
example_file = r".\test-data\movilidad_municipios_2023-01_origen.csv"
example_file_df = pd.read_csv(example_file)

example_file_date_df = utils.group_by_date(example_file_df)
pd.concat([example_file_date_df, example_file_date_df])

,day,day_of_week,viajes
0,2023-01-01,Domingo,2169439
1,2023-01-02,Lunes,3465322
2,2023-01-03,Martes,3671647
3,2023-01-04,Miércoles,3917849
4,2023-01-05,Jueves,3600172
...,...,...,...
26,2023-01-27,Viernes,4117152
27,2023-01-28,Sábado,3001308
28,2023-01-29,Domingo,2327012
29,2023-01-30,Lunes,3624180


In [43]:
day_df = pd.read_csv(file_paths[0])
day_df["date"] = pd.to_datetime(day_df["day"])
day_df["week"] = (day_df["day_of_week"] == "Lunes").cumsum()
day_df["year"] = day_df["date"].dt.year
day_df["month"] = day_df["date"].dt.month
day_df["day"] = day_df["date"].dt.day
day_df = day_df[["date", "year", "month", "day", "week", "day_of_week", "viajes"]]
day_df

,date,year,month,day,week,day_of_week,viajes
0,2023-01-01,2023,1,1,0,Domingo,2169439
1,2023-01-02,2023,1,2,1,Lunes,3465322
2,2023-01-03,2023,1,3,1,Martes,3671647
3,2023-01-04,2023,1,4,1,Miércoles,3917849
4,2023-01-05,2023,1,5,1,Jueves,3600172
...,...,...,...,...,...,...,...
966,2025-08-27,2025,8,27,139,Miércoles,3465719
967,2025-08-28,2025,8,28,139,Jueves,3412957
968,2025-08-29,2025,8,29,139,Viernes,3705710
969,2025-08-30,2025,8,30,139,Sábado,3039500
